#### Sklearn: Working with Text Data
https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
# Import dependencies (packages and modules)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV

#### Load the data set into jupyter notebook
The endstate is a dataframe containing a label column and text column

In [2]:
# get csv file from AWS S3 bucket
url ="https://wine-demo-bucket.s3.amazonaws.com/wine_variety.csv"

df = pd.read_csv(url)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131515 entries, 0 to 131514
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   country             131515 non-null  object 
 1   winery              131515 non-null  object 
 2   variety             131515 non-null  object 
 3   province            131515 non-null  object 
 4   region_1            131515 non-null  object 
 5   name                131515 non-null  object 
 6   points              131515 non-null  int64  
 7   price               131515 non-null  float64
 8   points_earned       131515 non-null  int64  
 9   value_points_per_$  131515 non-null  float64
 10  description         131515 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 11.0+ MB


In [3]:
# reduce the dataframe to just wine_type and description
# this is going to be our 'label' and 'text' columns (in the cell below)
column_names = ['variety', 'description']
df = df.reindex(columns=column_names)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131515 entries, 0 to 131514
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   variety      131515 non-null  object
 1   description  131515 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [4]:
# rename wine type to 'label' and description to 'text'
df.rename(columns={"variety": 'label', "description": 'text'}, inplace=True)
data = df
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131515 entries, 0 to 131514
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   131515 non-null  object
 1   text    131515 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


In [5]:
# how many of each wine_type are in our wine_description dataframe?
data['label'].value_counts()

Chardonnay                 13351
Pinot Noir                 13246
Cabernet Sauvignon         11891
Red Blend                   8787
Sauvignon Blanc             5970
                           ...  
Malvoisie                      1
Cabernet Franc-Malbec          1
Vidadillo                      1
Chardonnay-Pinot Gris          1
Chardonnay-Pinot Grigio        1
Name: label, Length: 543, dtype: int64

In [6]:
data['text'].head()

0    Ripe aromas of fig, blackberry and cassis are ...
1    Mac Watson honors the memory of a wine once ma...
2    This spent 20 months in 30% new French oak, an...
3    This is the top wine from La Bégude, named aft...
4    Deep, dense and pure from the opening bell, th...
Name: text, dtype: object

#### Devide the data into train and test using sklearn train_test_split

In [7]:
X = data['text'] # the description
y = data['label'] # the wine type

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### Extract features by tokenizing text using the bag of words technique


In [8]:
# we will tokenize and filter stop words using CountVectorizer
# this will be used to build a dictionary of features and transforms documents to feature vectors:
count_vect = CountVectorizer()
count_vect

CountVectorizer()

In [9]:
# creating bag of words for training data
# Here by doing ‘count_vect.fit_transform()’
# we are learning the vocabulary dictionary and it returns a Document-Term matrix. [n_samples, n_features].
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(98636, 26862)

#### CountVectorizer supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [10]:
# CountVectorizer supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:
count_vect.vocabulary_.get(u'algorithm')

#### From occurrences to frequencies
From occurrences to frequencies¶Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

#### TF - Term Frequencies#### TF - Term Frequencies
To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

#### TF - IDF (Term Frequency times inverse document frequency)#### TF - IDF (Term Frequency times inverse document frequency)
Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

Both tf and tf–idf can be computed as follows using TfidfTransformer:

In [11]:
# for our calculations we need to calculate TF and TF-IDF
# We can achieve both using below line of code and the X_train_counts from above:

tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

# The output will be the dimension of the Document-Term matrix -> (11314, 130107) which is the same shape as our X_train_counts shape

(98636, 26862)

#### Training a Classifier on the X_train_tfidf and y_train data
Naive Bayes Classifier: the one most suitable for word counts is the multinomial variant

In [12]:
# from sklearn.naive_bayes import MultinomialNB - this was done under dependencies in cell [1]
clf = MultinomialNB().fit(X_train_tfidf, y_train)

#### After fitting the training set, let’s make a prediction using the first row of X_test and y_test.

To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:

In [13]:
# we are going to want the text from a row (take first row) of X_test
new_description = [X_test.iloc[0]]

new_description

["Here's the latest in a string of great Cabs from Vine Cliff. It shows the full ripeness of the vintage, with a rich array of blackberry, cassis, cherry, chocolate and caramel flavors. But it wisely preserves structure with fine acidity and a complex tannic architecture. Should develop well over the next 10 years."]

In [14]:
# if we enter this into a prediction it should produce the y_test.iloc[0] category
y_test.iloc[0]

'Cabernet Sauvignon'

####  Build a pipeline
In order to make the vectorizer => transformer => classifier easier to work with, scikit-learn provides a Pipeline class that behaves like a compound classifier:




In [15]:
# MultinomialNb
# The names vect, tfidf and clf (classifier) are arbitrary. 
# We will use them to perform grid search for suitable hyperparameters below. 
# We can now train the model with a single command:

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [16]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

#### Evaluation of the performance of the MultinomialNb model on the test set
Evaluating the predictive accuracy of the model is equally easy:

In [17]:
#>>> docs_test = twenty_test.data
#>>> predicted = text_clf.predict(docs_test)
#>>> np.mean(predicted == twenty_test.target)
#0.8348...

docs_test = data['text']
predicted = text_clf.predict(docs_test)
np.mean(predicted == data['label'])

0.37406379500437215

We achieved 37.4% accuracy. Let’s see if we can do better with a linear support vector machine (SVM), which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). We can change the learner by simply plugging a different classifier object into our pipeline:

In [18]:
# using Support Vector Machine classifier with Stochastic gradient descent (SGD) learning
# note there are many parameters for SGDClassifier - for the first iteration we will use default parameter values

text_clf = Pipeline ([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())
])

text_clf.fit(data['text'], data['label'])

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier())])

In [19]:
predicted = text_clf.predict(data['text'])
np.mean(predicted == data['label'])

0.7266091320381706

###### We achieved 72.6% accuracy using the SVM. scikit-learn provides further utilities for more detailed performance analysis of the results:

In [20]:
print(metrics.classification_report(data['label'], predicted))

C:\Users\jamie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jamie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                     precision    recall  f1-score   support

                          Aglianico       0.89      0.76      0.82       257
                              Airen       0.60      1.00      0.75         6
                             Albana       0.69      0.73      0.71        15
                           Albariño       0.91      0.66      0.76       529
                          Albarossa       0.00      0.00      0.00         1
                            Albarín       0.00      0.00      0.00         1
                           Aleatico       0.70      0.70      0.70        10
                        Alfrocheiro       0.71      0.94      0.81        18
                           Alicante       0.64      0.90      0.75        10
                  Alicante Bouschet       0.47      0.50      0.49        36
                            Aligoté       0.91      0.33      0.49        30
                 Alsace white blend       0.68      0.55      0.61        5

C:\Users\jamie\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
metrics.confusion_matrix(data['label'], predicted)

array([[ 195,    0,    0, ...,    0,    3,    0],
       [   0,    6,    0, ...,    0,    0,    0],
       [   0,    0,   11, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    0,    0,    0],
       [   1,    0,    1, ...,    0, 3015,    1],
       [   0,    0,    0, ...,    0,    0,  150]], dtype=int64)

In [ ]:
plot_confusion_matrix(text_clf, X_test, y_test)  

#### Parameter tuning using grid search

We’ve already encountered some parameters such as use_idf in the TfidfTransformer. Classifiers tend to have many parameters as well; e.g., MultinomialNB includes a smoothing parameter alpha and SGDClassifier has a penalty parameter alpha and configurable loss and penalty terms in the objective function (see the module documentation, or use the Python help function to get a description of these).

Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and with a penalty parameter of either 0.01 or 0.001 for the linear SVM:

In [ ]:
# parameter tuning of the SGDClassifier()
# note: sklearn.model_selection import GridSearchCV  was imported with dependencies
# note below (1e-2, 1e-3) is the same as 0.01 or 0.001
parameters = {
    'vect__ngram_range': [(1,1), (1,2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

Obviously, such an exhaustive search can be expensive. If we have multiple CPU cores at our disposal, we can tell the grid searcher to try these eight parameter combinations in parallel with the n_jobs parameter. If we give this parameter a value of -1, grid search will detect how many cores are installed and use them all:

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [ ]:
gs_clf = gs_clf.fit(data['text'], data['label'])

The object’s best_score_ and best_params_ attributes store the best mean score and the parameters setting corresponding to that score:

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
# A more detailed summary of the search is available at gs_clf.cv_results_.
# gs_clf.cv_results_

Using parameter tuning for the parameters above did not improve over the default parameters
93.88 vs. 96.6 for 96.6% accuracy using the SVM with default parameters
TODO: confirm default parameters are valid

#### Next Step would be to examine performance of using a label of the nine wine_styles
Note sklearn how to states:
if you have multiple labels per document, e.g categories, have a look at the Multiclass and multilabel section.

https://scikit-learn.org/stable/modules/multiclass.html#multiclass

#### Review article on towards data science on multi-class-text-classification
https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f